In [1]:
import pandas as pd
import os

pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
# Import Data
cvs_fp = r'C:\Users\iya\Dev\CVM\CVS\2022'
cvs_tour_fn = r'sandag_cvs_tour.xlsx'
tnc_tour_fn = r'sandag_tnc_tour.xlsx'
cvs_df_fn = r'cv_20221102_sandag_draftfinal.xlsx'
db_fn = r'SANDAG 2022 CV DataBase & Dictionaires_03_03_2023.xlsx'

establishment_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Establishment Data')
trip_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Trip Data')
tour_data = pd.read_excel(os.path.join(cvs_fp, cvs_tour_fn), sheet_name='sandag_cvs_tour')
veh_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Vehicle Data')

In [3]:
indDict = {
    'Agriculture/Mining': 'Industrial',
    'Construction': 'Industrial',
    'Education/Other public services': 'Service/Gov/Office/FA',
    'Industrial/Utilities': 'Transport',
    'Info/FIRE/Professional services': 'Service/Gov/Office/FA',
    'Leisure/Accommodations and Food': 'Service/Gov/Office/FA', 
    'Manufacturing': 'Industrial',
    'Medical/Health Services': 'Service/Gov/Office/FA', 
    'Retail': 'Retail', 
    'Transportation': 'Transport',
    'Wholesale': 'Wholesale'
}

model_weight_dict = {
    'FORD TRANSIT': 'DA',
    'DODGE SPRINTER': 'DA', 
    'CHEVROLET C 4500': 'MHD',
    'FORD F450 SUPER DUTY FLATBED': 'LHD', 
    'INTERNATIONAL BOX': 'MHD',
    'FREIGHTLINER BOX': 'MHD', 
    'INTERNATIONAL STAKE': 'MHD', 
    'PETERBILT TANKER': 'HHD',
    'KENWORTH M2': 'MHD',
    'KENWORTH TANKER': 'MHD',
    'ISUZU NPR': 'MHD',
    'CHEVROLET 3500 EXPRESS BOX TRUCK' :'LHD',
    'CHEVROLET EXPRESS 2500': 'DA',
    'NISSAN NV3500': 'DA', 
    'CHEVROLET EXPRESS VAN 3500': 'DA', 
    'GMC SAVANA': 'DA',
    'FORD E450 BOX TRUCK': 'DA', 
    'CHEVROLET CARGO VAN': 'DA',
    'FREIGHTLINER COLUMBIA': 'HHD',
    'FREIGHTLINER CASCADIA': 'HHD',
    'KENWORTH T680': 'HHD',
    'FREIGHTLINER SPRINTER 2500': 'DA'
}

def assignWeightClass_model(make, model):
    global model_weight_dict
    mm = str(make) + ' ' + str(model)
    mm = mm.strip().upper()
    return model_weight_dict[mm]

def assignWeightClass(make, model, c, w):
    if c in [1, 2]: return 'DA'
    if c in [3,5,6]:
        if w <= 8500: return 'DA'
        elif 8500 < w <= 14000: return 'LHD'
        elif (c in [5,6]) and 14000 <= w < 33000: return 'MHD'
        elif w > 1000000: return assignWeightClass_model(make, model)
    if c >=5 and w >= 33000: return 'HHD'
    if c in [7, 8]: return 'HHD'
    if c == 3 and w > 20000: return 'DA'
        #if w > 5000000: return assignWeightClass_model(make, model)
        #else: return 'HHD'

In [4]:
# Add Vehicle Classification
trip_data = trip_data.merge(
    veh_data[['vehicle_classification', 'id']],
    left_on='vehicle_id',
    right_on='id',
    validate='many_to_one',
    how='left'
    )

# Add Industry Group
trip_data = trip_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

# Exclude Buses
trip_data = trip_data[trip_data.vehicle_classification != 4]
tour_data = tour_data[tour_data.VehClass != 4]
veh_data = veh_data[veh_data.vehicle_classification != 4]


# Assign Tour IDs (unsorted):
tour_id = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

started, ended = False, False
for se in list(zip(start, end)):
    if se == (True, False): 
        started, ended = True, False
        counter += 1
    elif se == (False, True):
        started, ended = False, True
    #if ended and not started: counter += 1
    tour_id.append(counter)

tour_data['tour_id'] = tour_id

# Add Industry Group
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

# CVM Industry Group
tour_data['industry_cvm'] = tour_data.IndName.apply(lambda x: indDict[x.strip()])
indNum_CVMIndName_dict = tour_data[[
    'base_location_Industry Group', 'industry_cvm'
]].groupby('base_location_Industry Group').max().to_dict()['industry_cvm']
trip_data['industry_cvm'] = trip_data['base_location_Industry Group'].apply(
    lambda x: indNum_CVMIndName_dict[x]
)

# Assign CVM Vehicle Weight Class
veh_data['weight_class_cvm'] = veh_data.apply(lambda x:
                                         assignWeightClass(
                                             x['veh_make_other'],
                                             x['vehicle_model'],
                                             x['vehicle_classification'],
                                             x['vehicle_weight']
                                             ),axis=1)

# Assign CVM Vehicle Weight Class to tour_data
tour_data = tour_data.merge(veh_data[['weight_class_cvm', 'id']], 
                      left_on='vehicle_id', 
                      right_on='id', 
                      how='left')

tour_data['weighted_distance'] = tour_data.TRAVEL_DISTANCE.fillna(0)*tour_data['Most.Likely.Estimate.Weight.Factor']
tour_data['VehClass_IndName'] = tour_data.apply(
    lambda x: x['weight_class_cvm'][0]+x['industry_cvm'][0], axis=1
)

# Tour Starts
tour_data_starts = tour_data[tour_data.TOUR_START == True]

In [5]:
# Employee Counts by Industry
employees = establishment_data[['base_location_Industry Group', 'no_of_emp_work']]
employees = employees.groupby('base_location_Industry Group').sum()
employees

,no_of_emp_work
base_location_Industry Group,
1,1114
2,6618
3,1025
4,6040
5,2852
6,3785
7,2133
8,6264
9,8220


In [6]:
# Trip Counts by Industry 
trip = trip_data[trip_data.arrival_time.notna()]
trip[
    ['industry_cvm', 'Most Likely Estimate Weight Factor']
    ].groupby('industry_cvm').sum()

,Most Likely Estimate Weight Factor
industry_cvm,
Industrial,"284,982.09"
Retail,"81,389.15"
Service/Gov/Office/FA,"1,101,987.30"
Transport,"106,357.56"
Wholesale,"69,012.34"


In [7]:
# Trip by VehClass and Industry
tour = tour_data[tour_data.TOUR_START != True]
tour.pivot_table(index='industry_cvm',
                             columns='weight_class_cvm',
                             values='Most.Likely.Estimate.Weight.Factor', 
                             aggfunc='sum'
                             )

weight_class_cvm,DA,HHD,LHD,MHD
industry_cvm,,,,
Industrial,"108,779.06","141,636.12","22,278.12","12,288.78"
Retail,"34,949.86","7,000.18","29,605.65","9,833.46"
Service/Gov/Office/FA,"721,149.19","81,845.60","129,631.82","169,360.69"
Transport,"39,408.91","26,487.79","38,348.61","2,112.25"
Wholesale,"36,695.41","21,436.48","5,884.57","4,995.88"


In [8]:
# Number of Tours by Industry 
tour_data_starts.groupby('industry_cvm')['Most.Likely.Estimate.Weight.Factor'].sum()

industry_cvm
Industrial                         67,514.61
Retail                             18,768.58
Service/Gov/Office/FA             197,940.58
Transport                          17,208.11
Wholesale                          13,948.57
Name: Most.Likely.Estimate.Weight.Factor, dtype: float64

In [9]:
# VMT by Industry 
tour_data[['weighted_distance', 'industry_cvm']].groupby('industry_cvm').sum().reset_index()

,industry_cvm,weighted_distance
0,Industrial,"3,546,892.84"
1,Retail,"662,266.36"
2,Service/Gov/Office/FA,"8,203,616.86"
3,Transport,"795,065.97"
4,Wholesale,"1,054,351.58"


In [10]:
# VMT by Industry and VehClass
tour_data.pivot_table(index='industry_cvm',
                      columns='weight_class_cvm',
                      values='weighted_distance',
                      aggfunc='sum')

weight_class_cvm,DA,HHD,LHD,MHD
industry_cvm,,,,
Industrial,"1,195,761.83","1,972,403.62","225,634.12","153,093.27"
Retail,"268,926.38","110,235.27","203,916.83","79,187.89"
Service/Gov/Office/FA,"5,828,685.78","411,751.69","993,912.63","969,266.77"
Transport,"353,758.51","326,603.04","95,057.13","19,647.29"
Wholesale,"646,717.53","319,924.22","32,037.88","55,671.95"


In [11]:
# Figure 9. Tour Start Time by Industry 
pd.options.display.float_format = '{:20,.5f}'.format

tb = pd.DataFrame(tour_data_starts.groupby(['industry_cvm', 'DEPART_TOD'])['Most.Likely.Estimate.Weight.Factor'].sum().reset_index())
pd.crosstab(
    tb.industry_cvm, 
    tb.DEPART_TOD, 
    values=tb['Most.Likely.Estimate.Weight.Factor'], 
    aggfunc='sum', 
    normalize='index')

DEPART_TOD,AM Peak\n6:00am-8:59am,Early AM\n3:00am-5:59am,Evening\n7:00pm-2:59am,Midday\n9:00am-3:29pm,PM Peak\n3:30pm-6:59pm
industry_cvm,,,,,
Industrial,0.37920,0.12175,0.00939,0.41501,0.07464
Retail,0.26720,0.01228,0.04243,0.63928,0.03881
Service/Gov/Office/FA,0.35369,0.03072,0.01572,0.53549,0.06438
Transport,0.31838,0.03766,0.02880,0.51127,0.10389
Wholesale,0.34054,0.09375,0.00856,0.49703,0.06013


In [12]:
# Figure 10. Vehicle Type by Industry
pd.crosstab(
    tour_data_starts.industry_cvm, 
    tour_data_starts.weight_class_cvm, 
    values=tour_data_starts['Most.Likely.Estimate.Weight.Factor'], 
    aggfunc='sum', 
    normalize='index')

weight_class_cvm,DA,HHD,LHD,MHD
industry_cvm,,,,
Industrial,0.41501,0.44625,0.08224,0.05650
Retail,0.45582,0.12082,0.34574,0.07762
Service/Gov/Office/FA,0.72493,0.05039,0.12594,0.09874
Transport,0.40709,0.43077,0.13127,0.03087
Wholesale,0.52906,0.32169,0.08346,0.06579


In [13]:
# Figure 12. Avg Trip Length by Vehicle Class and Industry 
tour_data_wc = tour_data[tour_data.TRAVEL_DISTANCE.notna()]

sum = tour_data_wc[['VehClass_IndName','weighted_distance']].groupby('VehClass_IndName').sum().reset_index()
tripWeightSum = tour_data_wc[['VehClass_IndName','Most.Likely.Estimate.Weight.Factor']].groupby('VehClass_IndName').sum().reset_index()
sum = sum.merge(tripWeightSum, on='VehClass_IndName')
sum['AvgTripDist'] = sum['weighted_distance'] / sum['Most.Likely.Estimate.Weight.Factor']
sum


,VehClass_IndName,weighted_distance,Most.Likely.Estimate.Weight.Factor,AvgTripDist
0,DI,"1,195,761.83469","108,779.06019",10.99257
1,DR,"268,926.37959","34,949.85746",7.69463
2,DS,"5,828,685.77775","721,149.18852",8.08250
3,DT,"353,758.50721","39,408.91347",8.97661
4,DW,"646,717.52996","36,695.41065",17.62393
5,HI,"1,972,403.61821","141,636.12146",13.92585
6,HR,"110,235.26660","7,000.17890",15.74749
7,HS,"411,751.68719","81,845.59605",5.03083
8,HT,"326,603.04036","26,487.78987",12.33032
9,HW,"319,924.21768","21,436.47853",14.92429
